# Modelagem

Inicialmente, o conjunto de dados foi dividido em conjuntos de treinamento e teste por meio de amostragem estratificada. O conjunto de teste foi isolado antes de qualquer etapa de pré-processamento. 

Todas as etapas de transformação, balanceamento, seleção de algoritmo e ajuste de hiperparâmetros foram realizadas exclusivamente no conjunto de treinamento utilizando validação cruzada estratificada. O modelo final foi então treinado com todos os dados de treinamento e avaliado uma única vez no conjunto de teste.

### Fluxograma do Pipeline de Dados

```text
       ┌──────────────────────────────────────────┐
       │     Dataset original (desbalanceado)     │
       │            ~10.000 registros             │
       └──────────────────────────────────────────┘
                     │
                     │ (Split estratificado)
                     ▼
       ┌──────────────────────────┐      ┌──────────────────────────────────────────┐
       │     Conjunto de Teste    │      │         Conjunto de Treinamento          │
       │   (~20% | ~2.000 reg.)   │◄─────┤           (~80% | ~8.000 reg.)           │
       │  ❌ Sem pré-processamento │      └──────────────────────────────────────────┘
       │  ❌ Sem balanceamento     │                    │
       └─────────────┬────────────┘                    ▼
                     │                 ┌──────────────────────────┐
                     │                 │     Pré-processamento    │
                     │                 │ (scaler, encoder, etc.)  │
                     │                 │   ✔ fit apenas no treino │
                     │                 └──────────────────────────┘
                     │                                 │
                     │                                 ▼
                     │                 ┌──────────────────────────┐
                     │                 │      Balanceamento       │
                     │                 │          (Under)         │
                     │                 │  ❗ apenas no treino     │
                     │                 └──────────────────────────┘
                     │                                 │
                     │                                 ▼
                     │                 ┌──────────────────────────┐
                     │                 │     Validação Cruzada    │
                     │                 │    (Stratified K-Fold)   │
                     │                 │                          │
                     │                 │ • Comparar algoritmos    │
                     │                 │ • Ajustar hiperparâmetros│
                     │                 │ • Estimar desempenho     │
                     │                 └──────────────────────────┘
                     │                                 │
                     │                                 ▼
                     │                 ┌──────────────────────────┐
                     │                 │  Modelo final escolhido  │
                     │                 │ (melhor algoritmo +      │
                     │                 │   melhores parâmetros)   │
                     │                 └──────────────────────────┘
                     │                                 │
                     │                                 ▼
                     │                 ┌──────────────────────────┐
                     │                 │     Treinamento final    │
                     │                 │      (100% do treino)    │
                     │                 │      ❌ sem CV           │
                     │                 └──────────────────────────┘
                     │                                 │
                     ▼                                 ▼
       ┌──────────────────────────────────────────────────────────┐
       │                 Avaliação final no teste                 │
       │ • Métricas reais (Precisão, AUC, curva ROC,              |
       | sensibilidade e especificidade)                          │
       │ • Validação de Generalização                             │
       │ • Executado uma única vez                                │
       └──────────────────────────────────────────────────────────┘

### Separando conjunto de teste e de treinamento

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd

from utils.general import *
from utils.norm_test import *
from utils.pre_process import *
from utils.analise_estatistica  import *

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_path = catchPath('data/data.xlsx')
data_path

'/ML/data/data.xlsx'

In [4]:
df = pd.read_excel(data_path)
df.head()

,ID_Cliente,Status_Cliente,Idade,Genero,Numero_Dependentes,Nivel_Educacao,Estado_Civil,Faixa_Renda,Categoria_Cartao,Meses_Relacionamento,...,Contatos_12m,Limite_Credito,Saldo_Rotativo,Limite_Disponivel,Variacao_Valor_Transacoes,Valor_Total_Transacoes,Quantidade_Transacoes,Variacao_Qtd_Transacoes,Taxa_Utilizacao_Credito,classe
0,768805383,Cliente Ativo,45,Masculino,3,Ensino Médio,Casado,60k a 80k,Azul,39,...,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,1
1,818770008,Cliente Ativo,49,Feminino,5,Graduação,Solteiro,Menos de 40k,Azul,44,...,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,1
2,713982108,Cliente Ativo,51,Masculino,3,Graduação,Casado,80k a 120k,Azul,36,...,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,1
3,769911858,Cliente Ativo,40,Feminino,4,Ensino Médio,Não Informado,Menos de 40k,Azul,34,...,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,1
4,709106358,Cliente Ativo,40,Masculino,3,Sem Escolaridade,Casado,60k a 80k,Azul,21,...,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,1


In [5]:
df.shape

(10127, 22)

### Tabela Final de Decisão de Features

| Variável | Evidência Estatística | Correlação | Decisão |
|--|--|--|--|
| Quantidade_Transacoes | Forte | Alta com Valor_Total | Manter |
| Taxa_Utilizacao_Credito | Moderada | Baixa | Manter |
| Meses_Inativos_12m | Moderada | Baixa | Manter |
| Variacao_Qtd_Transacoes | Significativa | Baixa | Manter |
| Variacao_Valor_Transacoes | Significativa | Baixa | Manter |
| Valor_Total_Transacoes | Moderada | Alta | Substituir |
| Limite_Credito | Fraca | Moderada | Opcional |
| Idade | Não significativa | — | Remover |


In [6]:
df.columns

Index(['ID_Cliente', 'Status_Cliente', 'Idade', 'Genero', 'Numero_Dependentes',
       'Nivel_Educacao', 'Estado_Civil', 'Faixa_Renda', 'Categoria_Cartao',
       'Meses_Relacionamento', 'Total_Produtos', 'Meses_Inativos_12m',
       'Contatos_12m', 'Limite_Credito', 'Saldo_Rotativo', 'Limite_Disponivel',
       'Variacao_Valor_Transacoes', 'Valor_Total_Transacoes',
       'Quantidade_Transacoes', 'Variacao_Qtd_Transacoes',
       'Taxa_Utilizacao_Credito', 'classe'],
      dtype='object')

In [7]:
features = [
    'Quantidade_Transacoes',
    'Taxa_Utilizacao_Credito',
    'Meses_Inativos_12m',
    'Variacao_Qtd_Transacoes',
    'Variacao_Valor_Transacoes'    
]

In [8]:
X = df[features].copy()
X.shape

(10127, 5)

In [9]:
y = df['classe'].copy()
y.shape

(10127,)

In [10]:
# split ANTES do balanceamento
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=2
)

In [11]:
X_test.shape

(2026, 5)

In [12]:
X_test.head()

,Quantidade_Transacoes,Taxa_Utilizacao_Credito,Meses_Inativos_12m,Variacao_Qtd_Transacoes,Variacao_Valor_Transacoes
2671,39,0.000,0,0.345,0.604
1888,61,0.037,4,0.743,0.889
6971,49,0.000,4,1.227,1.007
9082,106,0.000,3,0.767,0.947
5576,68,0.217,1,0.700,0.646


In [13]:
teste = X_test.copy()

In [14]:
teste['classe'] = y_test.copy()

In [18]:
y_test

2671    0
1888    1
6971    1
9082    1
5576    1
       ..
2286    1
550     1
857     1
9507    1
377     1
Name: classe, Length: 2026, dtype: int64

In [16]:
teste.head()

,Quantidade_Transacoes,Taxa_Utilizacao_Credito,Meses_Inativos_12m,Variacao_Qtd_Transacoes,Variacao_Valor_Transacoes,classe
2671,39,0.000,0,0.345,0.604,0
1888,61,0.037,4,0.743,0.889,1
6971,49,0.000,4,1.227,1.007,1
9082,106,0.000,3,0.767,0.947,1
5576,68,0.217,1,0.700,0.646,1


In [17]:
teste.shape

(2026, 6)

In [20]:
SaveDataFrame(teste,'Grupo_teste')

In [21]:
treino = X_train.copy()

In [22]:
treino['classe'] = y_train.copy()

In [23]:
treino.head()

,Quantidade_Transacoes,Taxa_Utilizacao_Credito,Meses_Inativos_12m,Variacao_Qtd_Transacoes,Variacao_Valor_Transacoes,classe
2607,52,0.619,1,0.857,0.612,1
5418,50,0.000,3,0.923,0.602,0
2684,73,0.036,2,0.698,0.739,1
1345,42,0.000,1,0.750,0.782,1
4298,39,0.000,2,0.345,0.637,0


In [24]:
treino.shape

(8101, 6)

In [25]:
SaveDataFrame(treino,'Grupo_treino')

In [3]:
# determinando o nivel de confiança para a amostra de teste com um erro de 2%
df_validacao = calcular_confianca_amostral(N=10127, n=2026, e=0.02)
print(df_validacao)

                Parâmetro          Valor
0     População Total (N)         10.127
1  Tamanho da Amostra (n)  2.026 (20.0%)
2      Margem de Erro (e)           2.0%
3      Escore-Z calculado           2.01
4      Nível de Confiança          95.6%


A amostragem para construir o conjunto de dados para realizar os testes foi validado estatisticamente demonstrando representatividade. Para uma população total de $N = 10.127$, a amostra de teste selecionada de $n = 2.026$ (20% do dataset) resulta em um nível de confiança de aproximadamente 95,5%, com uma margem de erro estrita de apenas 2%.

# Pré Processamento

In [26]:
path_treino = catchPath('data/Grupo_treino.xlsx')
path_treino

'/ML/data/Grupo_treino.xlsx'

In [27]:
df2 = pd.read_excel(path_treino)
df2.head()

,Quantidade_Transacoes,Taxa_Utilizacao_Credito,Meses_Inativos_12m,Variacao_Qtd_Transacoes,Variacao_Valor_Transacoes,classe
0,52,0.619,1,0.857,0.612,1
1,50,0.000,3,0.923,0.602,0
2,73,0.036,2,0.698,0.739,1
3,42,0.000,1,0.750,0.782,1
4,39,0.000,2,0.345,0.637,0


In [28]:
df2.columns

Index(['Quantidade_Transacoes', 'Taxa_Utilizacao_Credito',
       'Meses_Inativos_12m', 'Variacao_Qtd_Transacoes',
       'Variacao_Valor_Transacoes', 'classe'],
      dtype='object')

In [29]:
col = ['Quantidade_Transacoes', 'Taxa_Utilizacao_Credito',
       'Meses_Inativos_12m', 'Variacao_Qtd_Transacoes',
       'Variacao_Valor_Transacoes']

In [31]:
x1 = df2[col].copy()

In [33]:
y1 = df2['classe'].copy()

In [34]:
# Normalizando dados
X1 = norm_Data(x1,col)
X1.head()

,Quantidade_Transacoes,Taxa_Utilizacao_Credito,Meses_Inativos_12m,Variacao_Qtd_Transacoes,Variacao_Valor_Transacoes
0,-0.542062,1.249765,-1.322628,0.604469,-0.673665
1,-0.626871,-0.996227,0.654055,0.880053,-0.719230
2,0.348433,-0.865604,-0.334287,-0.059436,-0.094980
3,-0.966107,-0.996227,-1.322628,0.157690,0.100952
4,-1.093321,-0.996227,-0.334287,-1.533389,-0.559750


In [36]:
y1.head()

0    1
1    0
2    1
3    1
4    0
Name: classe, dtype: int64

In [35]:
X1['classe'] = y1.copy()
X1.head()

,Quantidade_Transacoes,Taxa_Utilizacao_Credito,Meses_Inativos_12m,Variacao_Qtd_Transacoes,Variacao_Valor_Transacoes,classe
0,-0.542062,1.249765,-1.322628,0.604469,-0.673665,1
1,-0.626871,-0.996227,0.654055,0.880053,-0.719230,0
2,0.348433,-0.865604,-0.334287,-0.059436,-0.094980,1
3,-0.966107,-0.996227,-1.322628,0.157690,0.100952,1
4,-1.093321,-0.996227,-0.334287,-1.533389,-0.559750,0


In [37]:
SaveDataFrame(X1,'Grupo_treino_normalizado')

In [39]:
X1['classe'].value_counts()

classe
1    6799
0    1302
Name: count, dtype: int64

### Balanceamento

In [40]:
X1.head()

,Quantidade_Transacoes,Taxa_Utilizacao_Credito,Meses_Inativos_12m,Variacao_Qtd_Transacoes,Variacao_Valor_Transacoes,classe
0,-0.542062,1.249765,-1.322628,0.604469,-0.673665,1
1,-0.626871,-0.996227,0.654055,0.880053,-0.719230,0
2,0.348433,-0.865604,-0.334287,-0.059436,-0.094980,1
3,-0.966107,-0.996227,-1.322628,0.157690,0.100952,1
4,-1.093321,-0.996227,-0.334287,-1.533389,-0.559750,0


In [44]:
X1.shape

(8101, 6)

In [41]:
col

['Quantidade_Transacoes',
 'Taxa_Utilizacao_Credito',
 'Meses_Inativos_12m',
 'Variacao_Qtd_Transacoes',
 'Variacao_Valor_Transacoes']

In [42]:
x2 = X1[col].copy()
x2.head()

,Quantidade_Transacoes,Taxa_Utilizacao_Credito,Meses_Inativos_12m,Variacao_Qtd_Transacoes,Variacao_Valor_Transacoes
0,-0.542062,1.249765,-1.322628,0.604469,-0.673665
1,-0.626871,-0.996227,0.654055,0.880053,-0.719230
2,0.348433,-0.865604,-0.334287,-0.059436,-0.094980
3,-0.966107,-0.996227,-1.322628,0.157690,0.100952
4,-1.093321,-0.996227,-0.334287,-1.533389,-0.559750


In [45]:
x2.shape

(8101, 5)

In [46]:
y2 = X1['classe'].copy()
y2.head()

0    1
1    0
2    1
3    1
4    0
Name: classe, dtype: int64

In [47]:
X_resampled, y_resampled = balances_categories(x2, y2)

In [48]:
X_resampled.shape

(2604, 5)

In [49]:
y_resampled.value_counts()

classe
0    1302
1    1302
Name: count, dtype: int64

In [50]:
df_balance = X_resampled.copy()
df_balance.head()

,Quantidade_Transacoes,Taxa_Utilizacao_Credito,Meses_Inativos_12m,Variacao_Qtd_Transacoes,Variacao_Valor_Transacoes
1,-0.626871,-0.996227,0.654055,0.880053,-0.719230
4,-1.093321,-0.996227,-0.334287,-1.533389,-0.559750
13,-1.050916,-0.996227,0.654055,-2.088731,-1.644214
30,-0.754084,-0.996227,-0.334287,-1.015626,-1.790024
38,-0.881298,2.570510,-0.334287,-1.023977,0.410799


In [51]:
df_balance['classe'] = y_resampled
df_balance.head()

,Quantidade_Transacoes,Taxa_Utilizacao_Credito,Meses_Inativos_12m,Variacao_Qtd_Transacoes,Variacao_Valor_Transacoes,classe
1,-0.626871,-0.996227,0.654055,0.880053,-0.719230,0
4,-1.093321,-0.996227,-0.334287,-1.533389,-0.559750,0
13,-1.050916,-0.996227,0.654055,-2.088731,-1.644214,0
30,-0.754084,-0.996227,-0.334287,-1.015626,-1.790024,0
38,-0.881298,2.570510,-0.334287,-1.023977,0.410799,0


In [52]:
SaveDataFrame(df_balance,'Grupo_treino_normalizado_Balanceado')

In [54]:
df_balance.shape

(2604, 6)

In [55]:
# determinando o nivel de confiança para a amostra de teste com um erro de 2%
df_validacao = calcular_confianca_amostral(N=8101, n=2604, e=0.02)
print(df_validacao)

                Parâmetro          Valor
0     População Total (N)          8.101
1  Tamanho da Amostra (n)  2.604 (32.1%)
2      Margem de Erro (e)           2.0%
3      Escore-Z calculado           2.48
4      Nível de Confiança          98.7%


Optou-se pelo do metodo de under sampling, e obtivemos um grupo de treinamento  com classes balanceadas com um total de 2604 registros, essa nova subamostragem é representativa com uma margem de erro de 2% com nivel de confiança de 98,7%.